In [200]:
import sympy as sym

from sympy.abc import x, E, I, P, L, a, b, v, delta

from sympy.physics.continuum_mechanics.beam import Beam

In [228]:
I

Piecewise((I_0, (x > 0) & (x < -l_c + x_c)), (I_0*(l_c - x*((delta - 1)**3 + 1))/l_c, (x < x_c) & (x > -l_c + x_c)), (I_0*(-l_c*(delta - 1)**3 + x*((delta - 1)**3 + 1))/l_c, (x > x_c) & (x < l_c + x_c)), (I_0, (L > x) & (x > l_c + x_c)))

In [236]:

b.slope()

(R2*SingularityFunction(x, L, 2)/2 + R2*SingularityFunction(0, L, 3)/(6*L))*Piecewise((1/I_0, (x > 0) & (x < -l_c + x_c)), (l_c/(I_0*(l_c - x*((delta - 1)**3 + 1))), (x < x_c) & (x > -l_c + x_c)), (l_c/(I_0*(-l_c*(delta - 1)**3 + x*((delta - 1)**3 + 1))), (x > x_c) & (x < l_c + x_c)), (1/I_0, (L > x) & (x > l_c + x_c)))/E + (-R2*SingularityFunction(0, L, 3)/6 + R2*SingularityFunction(x, L, 3)/6 + R2*x*SingularityFunction(0, L, 3)/(6*L))*Piecewise((0, (x > 0) & (x < -l_c + x_c)), (l_c*((delta - 1)**3 + 1)/(I_0*(l_c - x*((delta - 1)**3 + 1))**2), (x < x_c) & (x > -l_c + x_c)), (l_c*(-(delta - 1)**3 - 1)/(I_0*(-l_c*(delta - 1)**3 + x*((delta - 1)**3 + 1))**2), (x > x_c) & (x < l_c + x_c)), (0, (L > x) & (x > l_c + x_c)))/E

In [263]:
sym.S.NegativeOne * sym.S(4) / 3 * sym.Abs(x - xc)

-4*Abs(x - x_c)/3

In [254]:
E = 30e9
I0 = 1.0
L = 20
def compute_RIL(xc = 1.0, delta = 0.0, depth = 1.0):
    P = -1e3
    
    lc = sym.S(3)/sym.S(2) * L
    
    Ic = I0 * (sym.S(1) - delta)**sym.S(3)
    r = (I0 - Ic) / lc

    I = sym.Piecewise(
        (I0, (x < (xc-lc)) & (x > 0)),
        (sym.simplify(I0 - r * x), (x > xc-lc) & (x < xc)),
        (sym.simplify(Ic + r * x), (x > xc) & (x < xc + lc)),
        (I0, (x > (xc+lc)) & (x < L)),
    )
    
    b = Beam(20, E, I)

    R1, R2 = sym.symbols ('R1, R2')

    b.apply_load(R1, 0, -1)
    b.apply_load(R2, L, -1)
    b.bc_deflection = [(0, 0), (L, 0)]
    
    b.apply_load(P, 0.333*L, -1)
    b.solve_for_reaction_loads(R1, R2)
    
    b.remove_load(P, 0.333*L, -1)
    
    b.apply_load(P, 0.999*L, -1)
    b.solve_for_reaction_loads(R1, R2)
    
    return b
    

In [255]:
compute_RIL().plot_loading_results()

KeyboardInterrupt: 

In [142]:
# ----------------
# Bending Moments
# ----------------

# For (0 <= x <= a)
M_left = P * b * x / L

# For (a <= x <= L)
M_right = P * b * x / L - P * (x - a)

# ----------------
# Diff. Equations
# ----------------
v = sym.symbols('v', cls=sym.Function) # vertical displacement

# Curvature
kappa_left = M_left # / (E * I)
kappa_right = M_right # / (E * I)

sym.pprint(kappa_left)
sym.pprint(kappa_right)

P⋅b⋅x
─────
  L  
              P⋅b⋅x
-P⋅(-a + x) + ─────
                L  


In [184]:
kappa = sym.Piecewise(
    (kappa_left, (x >= 0) & (x < a)),
    (kappa_right, (x >= a) & (x < L)))
kappa

Piecewise((P*b*x/L, (x >= 0) & (a > x)), (-P*(-a + x) + P*b*x/L, (a <= x) & (L > x)))

In [191]:
v_beam = sym.dsolve(E*I*sym.diff(v(x), x, 2)-kappa,
                    ics={v(0): 0, v(L): 0})

In [192]:
v_beam

Eq(v(x), Piecewise((P*b*x**3/(6*E*I*L), (x >= 0) & (a > x)), (P*a*x**2/(2*E*I) - P*x**3/(6*E*I) + P*b*x**3/(6*E*I*L), (a <= x) & (L > x))) - x*Piecewise((L**2*P*b/(6*E*I), (L >= 0) & (L < a)), ((-L**3*P + 3*L**2*P*a + L**2*P*b)/(6*E*I), (L >= a) & (L > L)))/L)

In [188]:
v_beam.subs(x, a)

Eq(v(a), C1 + C2*a + Piecewise((P*a**3*b/(6*E*I*L), (a >= 0) & (a > a)), (P*a**3/(3*E*I) + P*a**3*b/(6*E*I*L), (a <= a) & (L > a))))

In [157]:
v_left = sym.dsolve(E*I*sym.diff(v(x),x,2)-M_left)
v_left

Eq(v(x), C1 + C2*x + P*b*x**3/(6*E*I*L))

In [159]:
v_right = sym.dsolve(E*I*sym.diff(v(x),x,2)-M_right)
v_right

Eq(v(x), C1 + C2*x + P*a*x**2/(2*E*I) + P*x**3*(-1 + b/L)/(6*E*I))

In [160]:
theta_right = sym.diff(v_right.args[1], x)
theta_right

C2 + P*a*x/(E*I) + P*x**2*(-1 + b/L)/(2*E*I)

In [146]:
# Slope/Rotation
C1 = sym.symbols('C1')
C2 = sym.symbols('C2')
theta_left = sym.integrate(kappa_left, x)
theta_right = sym.integrate(kappa_right, x)
sym.pprint(theta_left)
sym.pprint(theta_right)

     2
P⋅b⋅x 
──────
 2⋅L  
         2             
        x ⋅(-L⋅P + P⋅b)
P⋅a⋅x + ───────────────
              2⋅L      


In [138]:
# Displacement
C3 = sym.symbols('C3')
C4 = sym.symbols('C4')
v_left = sym.integrate(theta_left, x) + C3
v_right = sym.integrate(theta_right, x) + C4
sym.pprint(v_left)
sym.pprint(v_right)

                 3
            P⋅b⋅x 
C₁⋅x + C₃ + ──────
             6⋅L  
                 2    3             
            P⋅a⋅x    x ⋅(-L⋅P + P⋅b)
C₂⋅x + C₄ + ────── + ───────────────
              2            6⋅L      


In [140]:
# Solve for constants

# At x=a the slopes (theta) are equal
C1 = sym.solve(theta_left.subs({'x':a, 'b': L-a})-theta_right.subs({'x':a, 'b': L-a}), C1)
C2 = sym.solve(theta_left.subs({'x':a, 'b': L-a})-theta_right.subs({'x':a, 'b': L-a}), C2)

In [141]:
C1

[C2 + P*a**2/2]

In [82]:
M_right

-P*(-a + x) + P*x*(L - a)/L

In [83]:
kappa_left

P*x*(L - a)/(E*I*L)

In [84]:
kappa_right

(-P*(-a + x) + P*x*(L - a)/L)/(E*I)

In [85]:
theta_left

C1 + x**2*(L*P - P*a)/(2*E*I*L)

In [86]:
theta_right

C2 + P*a*x/(E*I) - P*a*x**2/(2*E*I*L)

In [87]:
v_left

C1*x + C3 + x**3*(L*P - P*a)/(6*E*I*L)

In [88]:
v_right

C2*x + C4 + P*a*x**2/(2*E*I) - P*a*x**3/(6*E*I*L)